<a href="https://colab.research.google.com/github/coryroyce/emnist_letter_exploration_and_prediction/blob/main/streamlit_application/Streamlit_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streamlit App - Letter Classification

Updated: 12/6/2021

Goal: Create an app wrapper for the letter prediction model. 

# Install Streamlit

In [1]:
!pip install streamlit -q
!pip install streamlit-drawable-canvas
!pip install ipykernel>=5.1.2
!pip install pydeck

# Clear output for this cell
from IPython.display import clear_output
clear_output()

Note: Before running these files you need to load the mnsit.hdf5 file into Colab so that it can load the model.

In [2]:
model_name = 'model_conv_v01'
!unzip /content/{model_name}.zip

Archive:  /content/model_conv_v01.zip
   creating: model_conv_v01/
   creating: model_conv_v01/assets/
   creating: model_conv_v01/variables/
  inflating: model_conv_v01/variables/variables.data-00000-of-00001  
  inflating: model_conv_v01/variables/variables.index  
  inflating: model_conv_v01/keras_metadata.pb  
  inflating: model_conv_v01/saved_model.pb  


In [8]:
%%writefile app.py
import streamlit as st
from streamlit_drawable_canvas import st_canvas
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
import pandas as pd
import plotly.express as px

# Load in the pre-trained model
model = tf.keras.models.load_model('/content/model_conv_v01')

# Define some valiables and helper functions
labels_caps = [
               'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z',
              ]
labels_dict_caps = {key: value for key, value in enumerate(labels_caps)}

def create_probability_fig(model_prediction):
    df_temp = pd.DataFrame(model_prediction, columns=labels_dict_caps.values())
    df_temp = df_temp.transpose().reset_index()
    df_temp.columns = ['Label','Probability']
    fig = px.bar(df_temp, x='Label', y='Probability')
    return fig


# Create the Application
st.title('Letter and Didgit Prediction')

# Create a drawing canvas with desired properties
canvas_result = st_canvas(
    fill_color="#ffffff",
    stroke_width=10,
    stroke_color='#ffffff',
    background_color="#000000",
    height=150,
    width=150,
    drawing_mode='freedraw',
    key="canvas",
)

# Show that the resized image looks like
st.markdown("What the model see's as input:")
if canvas_result.image_data is not None:
    img = cv2.resize(canvas_result.image_data.astype('uint8'), (28, 28))
    img_rescaling = cv2.resize(img, (192, 192), interpolation=cv2.INTER_NEAREST)
    st.write('Input Image')
    st.image(img_rescaling)

# Generate the prediciton based on the users drawings
if st.button('Predict'):
    x_user_input = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    pred = model.predict(x_user_input.reshape(1, 28, 28, 1))
    pred_label = labels_dict_caps[pred.argmax()]
    st.write(f'Predicted Label: {pred_label}')

    # Create a plotly barchart of the predicted probablities
    fig = create_probability_fig(pred)
    st.plotly_chart(fig, use_container_width = True)


Overwriting app.py


Load an example app for reference

In [9]:
!streamlit run app.py & npx localtunnel --port 8501


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.236.172.134:8501

npx: installed 22 in 4.337s
your url is: https://wise-otter-53.loca.lt
2021-12-07 05:59:26.033768: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
  Stopping...
^C
